In [8]:
import bs4
import requests
import pandas as pd
import collections as col
import spacy
nlp = spacy.load('fr_core_news_sm')

In [9]:
def parse_soup(url):
    """
        The function parse the page with beautifulsoup
        @param :  string containing the url of the rss feed
        @return : object containing the parse page
    """
    req = requests.get(url)
    data = req.text
    soup = bs4.BeautifulSoup(data, "lxml")
    return(soup)

def get_contents():
    """
        We get the content of the different articles
        @return ! dictionnary with the sentences by articles
    """
    dictio = col.defaultdict(list)
    data = pd.read_csv('url_articles_jdle.csv',sep=';')
    try:
        for i,url in enumerate(data['0']):
            print(i,url)
            soup = parse_soup(url)
            # Content is in p tag
            for bal in soup.find_all('p'):
                if bal != None and len(bal.text) > 35 and '*' not in bal.text:
                    dictio[i].append(str(bal.text))
    except Exception as e:
        # Max retries exceeded with url (le serveur nous rejette si trop de tentative en peu de temps, 12 par 1 minutes)
        # Trouver une solution
        print(e)
    return dictio

def clean_contents(contents):
    """
        Remove the caracters with no sense in order to clean the content
    """
    data = {}
    for i in range(0,len(contents)):
        # Drop the cookies relatives information in the website
        contents[i] = contents[i][:-3]
        
        new_words = ''
        # List of words conerning an article
        words = ' '.join(contents[i])
        
        # Remove backslash caracter
        for word in words.split():
            if '\\' not in word:
                new_words = new_words + " " + word
                
        # Clean basic caracters
        contents[i] = nlp(new_words).text
        data[i] = contents[i]
    return data

In [11]:
clean_contents(get_contents())

0 http://www.journaldelenvironnement.net/article/une-plateforme-d-e-commerce-dit-adieu-aux-emballages-jetables,95839
1 http://www.journaldelenvironnement.net/article/du-photovoltaique-a-fessenheim-c-est-parti,95830
2 http://www.journaldelenvironnement.net/article/economie-circulaire-un-projet-de-loi-a-minima,95836
3 http://www.journaldelenvironnement.net/article/stocamine-l-etat-persiste-et-signe,95788
4 http://www.journaldelenvironnement.net/article/microplastiques-vers-une-interdiction-partielle-de-l-ue-en-2020,95770
5 http://www.journaldelenvironnement.net/article/un-nouveau-contrat-de-filiere-dedie-a-la-valorisation-des-dechets,95761
6 http://www.journaldelenvironnement.net/article/des-multinationales-s-allient-contre-les-dechets-plastiques,95698
7 http://www.journaldelenvironnement.net/article/dechets-de-chantier-la-charte-de-bonnes-intentions-des-artisans,95695
8 http://www.journaldelenvironnement.net/article/les-usines-de-desalinisation-polluent-plus-que-prevu,95650
9 http://www

{0: ' L’entreprise Terra Cycle a annoncé, ce 24 janvier, lors du Forum économique mondial de Davos, le lancement d’une plateforme d’e-commerce ayant essentiellement recours à des emballages réutilisés ou recyclés. Abonnez-vous au JDLE et soyez au cœur de l’information professionnelle en Hygiène, Sécurité et Environnement. Votre outil de veille environnementale 100 % numérique Prestations de services logistiques et prestations de traitement de dechets d’equipements electriques et electroniques professionnels ECO-SYSTEMES Moscou veut détruire une banque de semences unique La récupération d’eau de pluie pourrait coûter cher Micro-hydraulique: FNE s’inquiète du projet de relance... Les associations s’unissent pour défendre les éoliennes',
 1: ' Le gouvernement a lancé, mercredi 23 janvier, l’appel d’offres pour la réalisation des centrales solaires de Fessenheim. Abonnez-vous au JDLE et soyez au cœur de l’information professionnelle en Hygiène, Sécurité et Environnement. Votre outil de vei